In [4]:
using DifferentialEquations, CSV, DataFrames, Statistics, Plots
using BlackBoxOptim
using DASSL
using LSODA

In [5]:
const N=10 ## number of the channels

const T_a0=273.15+25 # Inlet air tempearture of the proces air
const T_an=273.15+60 # Inlet air humidity ratio of the process air
const Y_a0=0.0125 # Inlet air tempearture of the regen air
const Y_an=0.0125 # Inlet air humidity ratio of the regen air
const vec=6 # air velocity of process air and regen air

const W_0=0.001 # initial value of desiccant moisture content
const T_0=273.15+10 # initial value of desiccant tempearture

const cpv=2140 # specific heat of vapor
const cpa=1017 # specific heat of air, 50C
const rho_a=1 # density of air
const cpd=921 # specific heat of desiccant, regular density of silica gel
const cpl=4180 # specific heat of liquid water
const cpsp=900 # specific heat of supporting materials
const fd=0.005 # desiccant fraction in the wall or in the matrix
const fm=0.005 # silica gel fraction in the desiccant(silica gel+supporting materials)
const qst=2300000 # adsorption heat
"""
constant input: 
    Inlet states of two air streams
    moist air property
    desiccant property
    rotation speed
    air velocity
parameters：
    structure area, A
    structure perimeter, P
    wheel length, L
    heat and mass transfer coefficient, h
cost function:
    moisture uptake
    cost
"""

"constant input: \n    Inlet states of two air streams\n    moist air property\n    desiccant property\n    rotation speed\n    air velocity\nparameters：\n    structure area, A\n    structure perimeter, P\n    wheel length, L\n    heat and mass transfer coefficient, h\ncost function:\n    moisture uptake\n    cost\n"

In [6]:
# define the equilibrium of desiccant 
function equilibrium(W,T)
    RH=0.0078 - 0.05759*W + 24.16554*W^2 - 124.478*W^3 + 204.226*W^4
    Pvs= exp(23.296 - 3816.44/(T - 46.13))
    Y=0.62198*RH/(101325/Pvs - RH)
    return Y
end


equilibrium (generic function with 1 method)

In [7]:
# define the problem 
function system_dynamics!(du,u,p,t)
    A,P,L,h,hm=p
    u[1,1]=Y_a0
    u[1,2]=T_a0
    
    u[1,4]=W_0
    u[1,5]=T_0
    u[1,3]=equilibrium(u[1,4],u[1,5]) # desiccant layer 边界层空气的含湿量
    for i in 2:(N-1)
        ip, im = i+1, i-1 ## the index of the former channel and latter channel

        # air humidity ratio Ya 
        du[i,1]=-(u[ip,1]+u[im,1]-2*u[i,1])*vec/L*N+(u[i,3]-u[i,1])*P*hm/A/rho_a
        # air Temperature Ta
        du[i,2]=-(u[ip,2]+u[im,2]-2*u[i,2])*vec/L*N+(h*(u[i,5]-u[i,2])+cpv*hm*(u[i,5]-u[i,2])*(u[i,1]-u[i,3]))*P/A/rho_a/(cpa+u[i,1]*cpv)
    end
    u[N,1]=-(u[N-1,1]-u[N,1])*vec/L*N+(u[N,3]-u[N,1])*P*hm/A/rho_a
    u[N,2]=-(u[N-1,2]-u[N,2])*vec/L*N+(h*(u[N,5]-u[N,2])+cpv*hm*(u[N,5]-u[N,2])*(u[N,1]-u[N,3]))*P/A/rho_a/(cpa+u[N,1]*cpv)
    for i in 2:N
        ip, im = i+1, i-1 ## the index of the former channel and latter channel   
        # desiccant moisture content W
        du[i,4]=2*P*hm*(u[i,1]-u[i,3])/fd
        # desiccant Temperature Td
        du[i,5]=(h*(u[i,2]-u[i,5])+cpv*hm*(u[i,2]-u[i,5])*(u[i,1]-u[i,3])+hm*(u[i,1]-u[i,3])*qst)*P*2/(fd*(cpd+u[i,4]*cpl)+fm*cpsp)

        # desiccant humidity ratio Yd
        u[i,3]=equilibrium(u[i,4],u[i,5]) # should be u[i,3]=equilibrium(u[i,4],u[i,5])
    end
    
end


system_dynamics! (generic function with 1 method)

In [8]:
function Initialization_system(N)
    u=zeros(N,5)
    for i in 1:N
        u[i,1]=Y_a0
        u[i,2]=T_a0
        
        u[i,4]=W_0
        u[i,5]=T_0
        u[i,3]=equilibrium(W_0,T_0)
    end
    u
end

Initialization_system (generic function with 1 method)

cost_function (generic function with 1 method)

In [9]:

# Bounds for the parameters
#bounds = [(0.000001, 0.00001), (0.006, 0.01), (0.1, 0.25), (60,90), (0.05, 0.15)]

# Perform global optimization using Differential Evolution
#result = bboptimize(cost_function; SearchRange = bounds, Method = :adaptive_de_rand_1_bin_radiuslimited,
#PopulationSize = 150, MaxSteps = 30,  MaxTime = 300.0)
time=(0,200)
p=(0.0000025,0.0076,0.2,51,0.009)
prob_optimized = ODEProblem(system_dynamics!, Initialization_system(N), time, p)

solution_optimized = solve(prob_optimized, lsoda())

retcode: Success
Interpolation: 1st order linear
t: 38-element Vector{Float64}:
   0.0
   3.797742888891485e-8
   7.59548577778297e-8
   1.1393228666674455e-7
   1.519097155556594e-7
   1.8988714444457425e-7
   2.658420022224039e-7
   3.417968600002336e-7
   4.177517177780633e-7
   1.1773002955563603e-6
   1.936848873334657e-6
   2.6963974511129543e-6
   1.0291883228895925e-5
   ⋮
   0.04792233950655335
   0.06723738050446845
   0.2603877904836196
   0.4535382004627707
   0.6466886104419218
   2.578192710233433
   4.509696810024944
   6.441200909816455
  25.756241907731567
  45.07128290564668
  64.38632390356179
 200.0
u: 38-element Vector{Matrix{Float64}}:
 [0.0125 298.15 … 0.001 283.15; 0.0125 298.15 … 0.001 283.15; … ; 0.0125 298.15 … 0.001 283.15; 0.0125 298.15 … 0.001 283.15]
 [NaN 298.15 … 0.001 283.15; 0.01249998707756146 298.14991498383307 … 0.0010000000129222913 283.1500000649009; … ; NaN NaN … 0.0010000000129264674 283.15000006517636; -0.3402627591181934 -7016.765666191724 … 

In [13]:
[u[N, 2] - 273.15 for u in solution_optimized.u]  # 将温度转换为摄氏度


38-element Vector{Float64}:
    25.0
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
 -7289.915666191724
     ⋮
   NaN
   NaN
   NaN
   NaN
   NaN
   NaN
   NaN
   NaN
   NaN
   NaN
   NaN
   NaN

In [14]:
for i in solution_optimized[1,:]
    println(i)
end

0.0125
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


In [ ]:
solution_optimized[1,:]

In [34]:
function cost_function(params)
    u0=Initialization_system(N)
    tspan = (0, 100)
    prob = ODEProblem(system_dynamics!, u0, tspan, params)
    solution = solve(prob, lsoda())
    
    moiture_uptake = -sum((fill(Y_a0,length(solution[1,:])) - solution[1,:])*params[1]*vec)
    
    return fitness
end

┌ Error: Some Julia code in the VS Code extension crashed
└ @ VSCodeServer c:\Users\h_guan\.vscode\extensions\julialang.language-julia-1.47.2\scripts\error_handler.jl:15
ERROR: UndefVarError: `Base64EncodePipe` not defined

In [33]:
bounds = [(0.000001, 0.00001), (0.006, 0.01), (0.1, 0.25), (60,90), (0.05, 0.15)]
good_guess = [0.0000025,0.0076,0.2,51,0.009]
# Perform global optimization using Differential Evolution
result = bboptimize(cost_function,good_guess,SearchRange = bounds,  Method = :random_search)

UndefVarError: UndefVarError: `A` not defined